#### This notebook calculates Jet-ACRE values of AMIP runs for each model

In [2]:
import pandas as pd
import numpy as np
import xarray as xr
from sklearn.decomposition import PCA
import os
import pickle

In [11]:
save_directory = '../data'
with open(os.path.join(save_directory, 'sam_models_monthly_2000_2014.pkl'), 'rb') as file:
    sam_models_monthly = pickle.load(file)

with open(os.path.join(save_directory, 'climatological_jet_lat_models_monthly_2000_2014.pkl'), 'rb') as file:
    climatological_jet_lat_models_monthly = pickle.load(file)

In [ ]:
from scipy.stats import linregress
directory = '/bjerknes_raid5/CMIP6_xl7pd/amip_monthly/longwave/ACRE'
file_list = os.listdir(directory)
jet_ACRE_index_models = {}
for file in file_list:
    file_name = os.path.join(directory, file)
    model_name = file.split('.')[2]
    
    if model_name in sam_models_monthly.keys():
        print(model_name)
        dataset = xr.open_dataset(file_name).sel(time=slice('2000', '2014'))
        dayofyear_mean = dataset.groupby('time.dayofyear').mean(dim='time')
        # Broadcast and subtract the mean from the original dataset
        anomaly = dataset.groupby('time.dayofyear') - dayofyear_mean
        latitudes = dataset.variables['lat'][:]
        acre_data = np.squeeze(anomaly.variables['ACRE'][:])

        regression_coeffs = np.zeros(len(latitudes))

        for i, lat in enumerate(latitudes):
            if lat < 0:
                acre_data_lat_mean = np.nanmean(acre_data[:, i, :], axis=1)
                df = pd.DataFrame({
                    'sam index': sam_models_monthly[model_name],
                    'acre_data_lat_mean': acre_data_lat_mean
                }).dropna()

                if len(df) > 1:
                    slope, _, _, _, _ = linregress((df['sam index'] - df['sam index'].mean())/df['sam index'].std(), df['acre_data_lat_mean'])
                    regression_coeffs[i] = slope
                else:
                    regression_coeffs[i] = np.nan
        jet_ACRE_index_models[model_name] = regression_coeffs

In [ ]:
with open(os.path.join(save_directory, 'SAM_ACRE_index_models_monthly.pkl'), 'wb') as f:
    pickle.dump(jet_ACRE_index_models, f)